# 모델 비교 KoFinBERT vs KrFinBERT

In [1]:
# Load model directly
from transformers import AutoModelForSequenceClassification, AutoTokenizer, TextClassificationPipeline
# tokenizer = AutoTokenizer.from_pretrained("kwoncho/KoFinBERT")
tokenizer = AutoTokenizer.from_pretrained("snunlp/KR-FinBert")
# tokenizer = AutoTokenizer.from_pretrained("snunlp/KR-FinBert-SC") # 없는 듯.
# model = AutoModelForSequenceClassification.from_pretrained("kwoncho/KoFinBERT") # 분류 성능이 좋지 못함.
model = AutoModelForSequenceClassification.from_pretrained("snunlp/KR-FinBert-SC")
# model = AutoModelForSequenceClassification.from_pretrained("./datas/results/kr-finbert-sc-2class") # 출력 레이어 웨이트가 바껴서 정확도 떨어짐
# model = AutoModelForSequenceClassification.from_pretrained("./datas/results/kr-finbert-sc-3class") # 출력 레이어 웨이트가 바껴서 정확도 떨어짐

/home/alpaco/anaconda3/envs/moon_mys/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [2]:
text_classifier = TextClassificationPipeline(
            tokenizer = tokenizer,
            model     = model
        )

In [11]:
# text_classifier(text)
# display(text_classifier('전날 상승 이어 이날도 올라갔다. 11%가량 오르기도 했다. 이날 2.43% 상승으로 장을 마쳤다.'))
display(text_classifier(' 테스트하고 싶은 문장을 넣어봅시다.'))
# display(text_classifier('전날 하락 이어 이날도 내려갔다. 11%가량 내리기도 했다. 이날 2.43% 하락으로 장을 마쳤다.'))

[{'label': 'positive', 'score': 0.9998836517333984}]

In [ ]:
# kwoncho/KoFinBERT: 해석이 어려움.
# [{'label': 'neutral', 'score': 0.9986956715583801}]
# [{'label': 'neutral', 'score': 0.9977371692657471}]

# snunlp/KR-FinBert-SC : 해석이 가능함.
# [{'label': 'positive', 'score': 0.9897533655166626}]
# [{'label': 'negative', 'score': 0.9925410747528076}]

결론 : KrFinBERT를 쓰자

# 레이어 분석

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import pandas as pd
import numpy as np

model_temp_1 = AutoModelForSequenceClassification.from_pretrained("snunlp/KR-FinBert-SC")
model_temp_2 = AutoModelForSequenceClassification.from_pretrained("snunlp/KR-FinBert-SC", num_labels=2, ignore_mismatched_sizes=True)


/home/alpaco/anaconda3/envs/moon_mys/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at snunlp/KR-FinBert-SC and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([3, 768]) in the checkpoint and torch.Size([2, 768]) in the model instantiated
- classifier.bias: found shape torch.Size([3]) in the checkpoint and torch.Size([2]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# 기본 모델 모든 레이어와 웨이트 확인하기
for name, param in model_temp_1.named_parameters():
    print(name, param.shape)

bert.embeddings.word_embeddings.weight torch.Size([20000, 768])
bert.embeddings.position_embeddings.weight torch.Size([512, 768])
bert.embeddings.token_type_embeddings.weight torch.Size([2, 768])
bert.embeddings.LayerNorm.weight torch.Size([768])
bert.embeddings.LayerNorm.bias torch.Size([768])
bert.encoder.layer.0.attention.self.query.weight torch.Size([768, 768])
bert.encoder.layer.0.attention.self.query.bias torch.Size([768])
bert.encoder.layer.0.attention.self.key.weight torch.Size([768, 768])
bert.encoder.layer.0.attention.self.key.bias torch.Size([768])
bert.encoder.layer.0.attention.self.value.weight torch.Size([768, 768])
bert.encoder.layer.0.attention.self.value.bias torch.Size([768])
bert.encoder.layer.0.attention.output.dense.weight torch.Size([768, 768])
bert.encoder.layer.0.attention.output.dense.bias torch.Size([768])
bert.encoder.layer.0.attention.output.LayerNorm.weight torch.Size([768])
bert.encoder.layer.0.attention.output.LayerNorm.bias torch.Size([768])
bert.encoder

In [ ]:
# num_labels = 2 수정 후 레이어 확인
for name, param in model_temp_2.named_parameters():
    print(name, param.shape)

# 클래스분류 층만 바뀜

bert.embeddings.word_embeddings.weight torch.Size([20000, 768])
bert.embeddings.position_embeddings.weight torch.Size([512, 768])
bert.embeddings.token_type_embeddings.weight torch.Size([2, 768])
bert.embeddings.LayerNorm.weight torch.Size([768])
bert.embeddings.LayerNorm.bias torch.Size([768])
bert.encoder.layer.0.attention.self.query.weight torch.Size([768, 768])
bert.encoder.layer.0.attention.self.query.bias torch.Size([768])
bert.encoder.layer.0.attention.self.key.weight torch.Size([768, 768])
bert.encoder.layer.0.attention.self.key.bias torch.Size([768])
bert.encoder.layer.0.attention.self.value.weight torch.Size([768, 768])
bert.encoder.layer.0.attention.self.value.bias torch.Size([768])
bert.encoder.layer.0.attention.output.dense.weight torch.Size([768, 768])
bert.encoder.layer.0.attention.output.dense.bias torch.Size([768])
bert.encoder.layer.0.attention.output.LayerNorm.weight torch.Size([768])
bert.encoder.layer.0.attention.output.LayerNorm.bias torch.Size([768])
bert.encoder

In [ ]:
# 특정 레이어의 웨이트 확인하기

# display(print(model_temp_1.state_dict()['bert.pooler.dense.weight']))
print(model_temp_1.state_dict()['classifier.weight'])
# print(model_temp_1.state_dict()['classifier.bias'])
print(' ')
# display(print(model_temp_2.state_dict()['bert.pooler.dense.weight']))
print(model_temp_2.state_dict()['classifier.weight'])
# print(model_temp_2.state_dict()['classifier.bias'])

# 클래스 분류층 웨이트만 바뀜

tensor([[ 0.0153,  0.0063,  0.0179,  ...,  0.0096, -0.0013,  0.0274],
        [-0.0050,  0.0316, -0.0152,  ..., -0.0051,  0.0118, -0.0021],
        [-0.0135,  0.0362,  0.0139,  ..., -0.0059,  0.0082, -0.0135]])
 
tensor([[ 0.0087,  0.0139,  0.0042,  ..., -0.0224,  0.0005, -0.0258],
        [-0.0177,  0.0553, -0.0382,  ...,  0.0185, -0.0184,  0.0461]])
